In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

In [4]:
print("reading csv")
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

print("contacting API")
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

reading csv
contacting API


{'companyName': 'Apple Inc',
 'marketcap': 2376941137420,
 'week52high': 161.78,
 'week52low': 107.44,
 'week52highSplitAdjustOnly': 160.97,
 'week52lowSplitAdjustOnly': 110.41,
 'week52change': 0.25097136874843,
 'sharesOutstanding': 17303754186,
 'float': 0,
 'avg10Volume': 91144717,
 'avg30Volume': 91918732,
 'day200MovingAvg': 142.15,
 'day50MovingAvg': 152.63,
 'employees': 153438,
 'ttmEPS': 5.3,
 'ttmDividendRate': 0.869256523676388,
 'dividendYield': 0.00599976936559118,
 'nextDividendDate': '',
 'exDividendDate': '2021-08-04',
 'nextEarningsDate': '2021-10-22',
 'peRatio': 27.95391961167444,
 'beta': 1.459334873779919,
 'maxChangePercent': 55.13194763167478,
 'year5ChangePercent': 4.475227475973166,
 'year2ChangePercent': 1.562506291182465,
 'year1ChangePercent': 0.2702607229222612,
 'ytdChangePercent': 0.07791157577081008,
 'month6ChangePercent': 0.12946191281994404,
 'month3ChangePercent': 0.001422201329562216,
 'month1ChangePercent': -0.08292453750732535,
 'day30ChangePerce

In [6]:
data['year1ChangePercent']

print("defining group API call logic")
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])
print("done")

defining group API call logic
done


In [7]:
print("creating pandas dataframe")
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']


final_dataframe = pd.DataFrame(columns = my_columns)
print("done")

creating pandas dataframe
done


In [8]:
print("populating pandas dataframe with API data (SLOOOOOOW)")
for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

populating pandas dataframe with API data (SLOOOOOOW)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,161.75,0.525338,N/A
1,AAL,20.46,0.661469,N/A
2,AAP,218.30,0.379979,N/A
3,AAPL,144.30,0.272385,N/A
4,ABBV,112.30,0.342904,N/A
...,...,...,...,...
500,YUM,130.10,0.374433,N/A
501,ZBH,149.38,0.0454006,N/A
502,ZBRA,517.25,0.889475,N/A
503,ZION,63.77,1.09539,N/A


In [9]:
print("sorting dataframe for right values, then cutting everything below 50th place off")
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

sorting dataframe for right values, then cutting everything below 50th place off


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,DVN,40.00,3.09969,N/A
1,MRO,16.13,2.91135,N/A
2,FANG,105.75,2.75252,N/A
3,LB,80.84,2.33433,N/A
4,OXY,34.03,2.19023,N/A
5,MCHP,154.63,1.82285,N/A
6,COTY,8.34,1.74818,N/A
7,SIVB,687.45,1.6704,N/A
8,FTNT,309.74,1.57021,N/A
9,EOG,89.98,1.51735,N/A


In [11]:
print("portfolio input logic...")
# def portfolio_input():
#     global portfolio_size
#     portfolio_size = input("Enter the value of your portfolio:")

#     try:
#         val = float(portfolio_size)
#     except ValueError:
#         print("That's not a number! \n Try again:")
#         portfolio_size = input("Enter the value of your portfolio:")

# portfolio_input()

portfolio_size = 10000000

print(portfolio_size)
print("done")

portfolio input logic...
10000000
done


In [12]:
print("creating a more refined high level selection of high value stocks (stocks that have been accumulating value over a longer perdiod of time)")
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

creating a more refined high level selection of high value stocks (stocks that have been accumulating value over a longer perdiod of time)


/Users/henribakker/code/projects/algorithmic_trading/.venv/lib/python3.9/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,DVN,40.00,3.09969,4901
1,MRO,16.13,2.91135,12156
2,FANG,105.75,2.75252,1854
3,LB,80.84,2.33433,2425
4,OXY,34.03,2.19023,5761
5,MCHP,154.63,1.82285,1268
6,COTY,8.34,1.74818,23510
7,SIVB,687.45,1.6704,285
8,FTNT,309.74,1.57021,633
9,EOG,89.98,1.51735,2179


In [13]:
print("creating hqm dataframe")
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

print("done")

creating hqm dataframe
done


In [14]:
print("populating dataframe with API data. Will take a while.")

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns
hqm_dataframe

populating dataframe with API data. Will take a while.


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,158.23,N/A,0.544426,N/A,0.177016,N/A,0.0333168,N/A,-0.14565,N/A,N/A
1,AAL,20.43,N/A,0.650469,N/A,-0.147518,N/A,-0.023468,N/A,0.0616119,N/A,N/A
2,AAP,215.90,N/A,0.382673,N/A,0.16874,N/A,0.0176643,N/A,0.0662871,N/A,N/A
3,AAPL,149.32,N/A,0.273762,N/A,0.130873,N/A,0.00140332,N/A,-0.0800089,N/A,N/A
4,ABBV,113.17,N/A,0.345776,N/A,0.0645425,N/A,-0.0470582,N/A,-0.0215673,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,130.58,N/A,0.3859,N/A,0.0945962,N/A,0.0702927,N/A,-0.0563927,N/A,N/A
501,ZBH,153.09,N/A,0.0456762,N/A,-0.102469,N/A,-0.102024,N/A,-0.00114165,N/A,N/A
502,ZBRA,514.30,N/A,0.876378,N/A,0.0314764,N/A,-0.0697747,N/A,-0.143762,N/A,N/A
503,ZION,64.67,N/A,1.14245,N/A,0.152078,N/A,0.228865,N/A,0.103506,N/A,N/A


In [84]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

start = 401
finish = 504

print("Ticker, Price, Number of Shares to Buy, One-Year Price Return")
for i in range(start, finish):
    row = i
    print(f"row {row}:{type(hqm_dataframe['Ticker'][row])}, {type(hqm_dataframe['Price'][row])}, {type(hqm_dataframe['Number of Shares to Buy'][row])}, {type(hqm_dataframe['One-Year Price Return'][row])}")

print("\n\n")

print("One-Year Return Percentile, Six-Month Price Return, Six-Month Return Percentile")  
for i in range(start, finish):
    row = i
    print(f"row {row}:{type(hqm_dataframe['One-Year Return Percentile'][row])}, {type(hqm_dataframe['Six-Month Price Return'][row])}, {type(hqm_dataframe['Six-Month Return Percentile'][row])}")

print("\n\n")       
    
print("Three-Month Price Return, Three-Month Return Percentile, One-Month Price Return, One-Month Return Percentile")  
for i in range(start, finish):
    row = i
    print(f"row {row}:{type(hqm_dataframe['Three-Month Price Return'][row])}, {type(hqm_dataframe['Three-Month Return Percentile'][row])}, {type(hqm_dataframe['One-Month Price Return'][row])}, {type(hqm_dataframe['One-Month Return Percentile'][row])}")






Ticker, Price, Number of Shares to Buy, One-Year Price Return
row 401:<class 'str'>, <class 'numpy.float64'>, <class 'str'>, <class 'float'>
row 402:<class 'str'>, <class 'numpy.float64'>, <class 'str'>, <class 'float'>
row 403:<class 'str'>, <class 'numpy.float64'>, <class 'str'>, <class 'float'>
row 404:<class 'str'>, <class 'numpy.float64'>, <class 'str'>, <class 'float'>
row 405:<class 'str'>, <class 'numpy.float64'>, <class 'str'>, <class 'float'>
row 406:<class 'str'>, <class 'numpy.float64'>, <class 'str'>, <class 'float'>
row 407:<class 'str'>, <class 'numpy.float64'>, <class 'str'>, <class 'float'>
row 408:<class 'str'>, <class 'numpy.float64'>, <class 'str'>, <class 'float'>
row 409:<class 'str'>, <class 'numpy.float64'>, <class 'str'>, <class 'float'>
row 410:<class 'str'>, <class 'numpy.float64'>, <class 'str'>, <class 'float'>
row 411:<class 'str'>, <class 'numpy.float64'>, <class 'str'>, <class 'float'>
row 412:<class 'str'>, <class 'numpy.float64'>, <class 'str'>, <class

In [83]:
#testcase
print(type(hqm_dataframe['One-Year Price Return'][100]))
print(type(hqm_dataframe['Six-Month Price Return'][100]))
print(type(hqm_dataframe['Three-Month Price Return'][100]))
print(type(hqm_dataframe['One-Month Price Return'][100]))

#known to return NoneType
print(type(hqm_dataframe['One-Year Price Return'][118]))
print(type(hqm_dataframe['Six-Month Price Return'][118]))
print(type(hqm_dataframe['Three-Month Price Return'][118]))
print(type(hqm_dataframe['One-Month Price Return'][118]))

print(type(hqm_dataframe['One-Year Price Return'][165]))
print(type(hqm_dataframe['Six-Month Price Return'][165]))
print(type(hqm_dataframe['Three-Month Price Return'][165]))
print(type(hqm_dataframe['One-Month Price Return'][165]))

print(type(hqm_dataframe['One-Year Price Return'][326]))
print(type(hqm_dataframe['Six-Month Price Return'][326]))
print(type(hqm_dataframe['Three-Month Price Return'][326]))
print(type(hqm_dataframe['One-Month Price Return'][326]))

print(type(hqm_dataframe['One-Year Price Return'][327]))
print(type(hqm_dataframe['Six-Month Price Return'][327]))
print(type(hqm_dataframe['Three-Month Price Return'][327]))
print(type(hqm_dataframe['One-Month Price Return'][327]))

<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>
<class 'NoneType'>


In [86]:
hqm_dataframe = hqm_dataframe.fillna(value=np.nan)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,158.230,N/A,0.544426,N/A,0.177016,N/A,0.033317,N/A,-0.145650,N/A,N/A
1,AAL,20.430,N/A,0.650469,N/A,-0.147518,N/A,-0.023468,N/A,0.061612,N/A,N/A
2,AAP,215.900,N/A,0.382673,N/A,0.168740,N/A,0.017664,N/A,0.066287,N/A,N/A
3,AAPL,149.320,N/A,0.273762,N/A,0.130873,N/A,0.001403,N/A,-0.080009,N/A,N/A
4,ABBV,113.170,N/A,0.345776,N/A,0.064542,N/A,-0.047058,N/A,-0.021567,N/A,N/A
5,ABC,124.270,N/A,0.241791,N/A,0.015606,N/A,0.016694,N/A,-0.067904,N/A,N/A
6,ABMD,348.200,N/A,0.281111,N/A,0.043986,N/A,0.033632,N/A,-0.095138,N/A,N/A
7,ABT,121.780,N/A,0.117323,N/A,-0.022715,N/A,-0.016413,N/A,-0.098495,N/A,N/A
8,ACN,330.200,N/A,0.500189,N/A,0.165425,N/A,0.069624,N/A,-0.053682,N/A,N/A
9,ADBE,587.190,N/A,0.193223,N/A,0.168580,N/A,-0.045756,N/A,-0.148444,N/A,N/A


In [92]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:

        # THE PROBLEM LINE
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
# for time_period in time_periods:
#     print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe


# print("if you made it here then wow you fixed one problem ")

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,158.230,N/A,0.544426,0.706931,0.177016,0.784158,0.033317,0.6,-0.145650,0.0257426,N/A
1,AAL,20.430,N/A,0.650469,0.784158,-0.147518,0.0772277,-0.023468,0.360396,0.061612,0.851485,N/A
2,AAP,215.900,N/A,0.382673,0.534653,0.168740,0.772277,0.017664,0.532673,0.066287,0.865347,N/A
3,AAPL,149.320,N/A,0.273762,0.4,0.130873,0.689109,0.001403,0.469307,-0.080009,0.229703,N/A
4,ABBV,113.170,N/A,0.345776,0.487129,0.064542,0.548515,-0.047058,0.257426,-0.021567,0.578218,N/A
5,ABC,124.270,N/A,0.241791,0.360396,0.015606,0.409901,0.016694,0.526733,-0.067904,0.30495,N/A
6,ABMD,348.200,N/A,0.281111,0.411881,0.043986,0.487129,0.033632,0.60198,-0.095138,0.138614,N/A
7,ABT,121.780,N/A,0.117323,0.239604,-0.022715,0.291089,-0.016413,0.386139,-0.098495,0.128713,N/A
8,ACN,330.200,N/A,0.500189,0.675248,0.165425,0.760396,0.069624,0.750495,-0.053682,0.392079,N/A
9,ADBE,587.190,N/A,0.193223,0.312871,0.168580,0.770297,-0.045756,0.265347,-0.148444,0.0237624,N/A


In [94]:
print(type(hqm_dataframe['One-Month Price Return'][118]))

<class 'numpy.float64'>
